In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI006_IK15HGAE_data_PCON"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

input_shape = (224, 224, 3)

# Define data augmentation techniques
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

# Setup train, validation and test generators
train_set = train_datagen.flow_from_directory(
    os.path.join(train_dir, "Cam3P1L2\\obj2"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    os.path.join(train_dir, "Cam3P1L2\\obj2"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    os.path.join(test_dir, "Cam3P1L2\\obj2"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the ResNet50 model, pre-trained on ImageNet
base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')

# Freeze the base model layers during the initial training phase.
base_model.trainable = False

# Custom layers on top of the base model
inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='softmax')(x)  # 10 units for 10 different defect types

# This is the model we will train
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(
    train_set,
    epochs=10,
    validation_data=val_set
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

In [2]:
#ResNet
import tensorflow as tf
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI005_6_IK15HGAE_data_PCBCAM3"
train_dir = os.path.join(data_dir, "train")
#test_dir = os.path.join(data_dir, "test")

img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

# Setup train, validation and test generators
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

#test_datagen = ImageDataGenerator(rescale=1./255)
#test_set = test_datagen.flow_from_directory(
#    test_dir,
#    target_size=(img_size, img_size),
#    batch_size=batch_size,
#    class_mode='binary',
#    shuffle=False
#)

# Load the ResNet50 model, pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=(img_size, img_size, 3)))

# Freeze the layers of the base model
for layer in base_model.layers[:-10]: #
    layer.trainable = False

# Create the top layers to be trained with our data
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x) #added
predictions = Dense(1, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True)
]

# Train the model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

# Evaluate the model on test data
#test_loss, test_acc = model.evaluate(test_set)
#print("Test accuracy:", test_acc)

Found 380 images belonging to 2 classes.
Found 94 images belonging to 2 classes.
Epoch 1/30
24/24 [==============================] - 90s 3s/step - loss: 0.7691 - accuracy: 0.4974 - val_loss: 0.7553 - val_accuracy: 0.5000
Epoch 2/30
24/24 [==============================] - 76s 3s/step - loss: 0.7301 - accuracy: 0.4974 - val_loss: 0.6959 - val_accuracy: 0.5000
Epoch 3/30
24/24 [==============================] - 81s 3s/step - loss: 0.6971 - accuracy: 0.5500 - val_loss: 0.6942 - val_accuracy: 0.4362
Epoch 4/30
24/24 [==============================] - 80s 3s/step - loss: 0.7091 - accuracy: 0.5263 - val_loss: 0.7017 - val_accuracy: 0.4043
Epoch 5/30
24/24 [==============================] - 79s 3s/step - loss: 0.6838 - accuracy: 0.5553 - val_loss: 0.7073 - val_accuracy: 0.5000
Epoch 6/30
24/24 [==============================] - 75s 3s/step - loss: 0.7161 - accuracy: 0.5158 - val_loss: 0.7089 - val_accuracy: 0.5000
Epoch 7/30
24/24 [==============================] - 77s 3s/step - loss: 0.6944 

In [3]:
model.save("AIVI005_6_PCB.h5")

In [ ]:
#EfficientNetB0
import tensorflow as tf
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI006_IK15HGAE_data_ALL"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

# Setup train, validation and test generators
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the ResNet50 model, pre-trained on ImageNet
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create the top layers to be trained with our data
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=50,
    validation_data=val_set
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

In [5]:
#AlexNet
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI006_IK15HGAE_data_PCBCAM3"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Define the class names to be predicted
img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

test_set = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

model = tf.keras.models.Sequential([
    Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(3, strides=(2,2)),
    Conv2D(256, (5,5), padding='same', activation='relu'),
    MaxPooling2D(3, strides=(2,2)),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    MaxPooling2D(3, strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with categorical cross-entropy loss
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)


Found 273 images belonging to 2 classes.
Found 67 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/30
18/18 [==============================] - 62s 3s/step - loss: 0.7008 - accuracy: 0.4652 - val_loss: 0.6923 - val_accuracy: 0.5224
Epoch 2/30
18/18 [==============================] - 50s 3s/step - loss: 0.6926 - accuracy: 0.5128 - val_loss: 0.6943 - val_accuracy: 0.4776
Epoch 3/30
18/18 [==============================] - 51s 3s/step - loss: 0.6947 - accuracy: 0.4725 - val_loss: 0.6928 - val_accuracy: 0.5224
Epoch 4/30
18/18 [==============================] - 50s 3s/step - loss: 0.6936 - accuracy: 0.5275 - val_loss: 0.6936 - val_accuracy: 0.4776
Epoch 5/30
18/18 [==============================] - 49s 3s/step - loss: 0.6936 - accuracy: 0.5165 - val_loss: 0.6920 - val_accuracy: 0.5224
Epoch 6/30
18/18 [==============================] - 51s 3s/step - loss: 0.6921 - accuracy: 0.5165 - val_loss: 0.6955 - val_accuracy: 0.5224
Epoch 7/30
18/18 [=====================

KeyboardInterrupt: 

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir = "D:\\01.Paul\\overall_IK15HGAE_data2\\test\\Cam3P1L1\\obj2"

# Define the data generator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of test data
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

Found 248 images belonging to 2 classes.
16/16 [==============================] - 21s 1s/step - loss: 0.9087 - accuracy: 0.6169
Test accuracy: 0.6169354915618896


In [ ]:
#Fully Convolutional Networks (FCNs)
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

data_dir = "D:\\01.Paul\\AIVI005_IK15HGA\\data_5_15HE"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")


class_names = ["Cam3P1L1\\obj2", "Cam3P1L1\\obj2"]
img_size = 256

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    os.path.join(train_dir, class_names[0]),
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    os.path.join(train_dir, class_names[0]),
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

test_set = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(train_dir, class_names[1]),
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

model = tf.keras.models.Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(img_size, img_size, 3)),
    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(4096, kernel_size=(7, 7), activation='relu', padding='valid'),
    Conv2D(4096, kernel_size=(1, 1), activation='relu', padding='valid'),
    Conv2D(1, kernel_size=(1, 1), activation='sigmoid', padding='same'),
    UpSampling2D(size=(32, 32), interpolation='bilinear')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

Found 238 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 297 images belonging to 2 classes.
Epoch 1/30
15/15 [==============================] - 792s 53s/step - loss: 4.0831 - accuracy: 0.7226 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 2/30
15/15 [==============================] - 882s 59s/step - loss: 4.2288 - accuracy: 0.7227 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 3/30
15/15 [==============================] - 824s 55s/step - loss: 4.2288 - accuracy: 0.7227 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 4/30
15/15 [==============================] - 802s 54s/step - loss: 4.2288 - accuracy: 0.7227 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 5/30
15/15 [==============================] - 769s 51s/step - loss: 4.2288 - accuracy: 0.7227 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 6/30
15/15 [==============================] - 691s 46s/step - loss: 4.2288 - accuracy: 0.7227 - val_loss: 4.1354 - val_accuracy: 0.7288
Epoch 7/30
15/15 [========

In [ ]:
img_path = os.path.join(test_good_dir, "example.jpg")
img = cv2.imread(img_path)
img = cv2.resize(img, (img_size, img_size))
img = np.expand_dims(img, axis=0)
prob_map = model.predict(img)[0,:,:,1]  # probability map for the defective class
prob_map = cv2.resize(prob_map, (img

In [ ]:
import cv2

height, width = img.shape[:2]

In [7]:
img = cv2.imread("D:\\01.Paul\\AIVI_image_IK15HGA_KUUE00E\\data\\test\\good\\Cam1P1L1\\obj2\\Frame001B1Row1Col02Obj02.jpg")

In [9]:
height, width = img.shape[:2]

In [10]:
height

2000

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical


#Set up the data directory path containing images and labels

data_dir = "D:\01.Paul\AIVI_image_IK15HGA_KUUE00E\data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")


#Set up the file paths for good and defective training images

train_good_dir = os.path.join(train_dir, "good\Cam3P1L1\obj2")
train_defective_dir = os.path.join(train_dir, "defective\Cam3P1L1\obj2")


#Set up the file paths for good and defective test images

test_good_dir = os.path.join(test_dir, "good\Cam3P1L1\obj3")
test_defective_dir = os.path.join(test_dir, "defective\Cam3P1L1\obj2")


#Define the class names to be predicted

class_names = ["good\Cam3P1L1\obj2", "defective\Cam3P1L1\obj2"]


img_size = 224
batch_size = 16


#Define data augmentation techniques

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)


train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


test_set = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


#Define a function to extract patches from an image

def extract_patches(img, patch_size=(64, 64)):
    height, width, _ = img.shape
    patches = []
    for i in range(0, height - patch_size[0], patch_size[0]):
        for j in range(0, width - patch_size[1], patch_size[1]):
            patch = img[i:i+patch_size[0], j:j+patch_size[1], :]
            patches.append(patch)
    return patches


model = tf.keras.models.Sequential([
    Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(3, strides=(2,2)),
    Conv2D(256, (5,5), padding='same', activation='relu'),
    MaxPooling2D(3, strides=(2,2)),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    MaxPooling2D(3, strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])


#Compile the model with categorical cross-entropy loss

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])


#Train the model

history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)



#Evaluate the model on test data

test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)


#Extract patches from a test image and make predictions on each patch

img_path = os.path.join(test_good_dir, "example.jpg")
img = cv2.imread(img_path)
patches = extract_patches(img, patch_size=(224,224))
for patch in patches:
    patch = cv2.resize(patch, (img_size, img_size))
    patch = np.expand_dims(patch, axis=0)

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI_image_IK15HGA_KUUE00E\\data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Set up the file paths for good and defective training images
train_good_dir = os.path.join(train_dir, "good\\Cam3P1L1\\obj2")
train_defective_dir = os.path.join(train_dir, "defective\\Cam3P1L1\\obj2")

# Set up the file paths for good and defective test images
test_good_dir = os.path.join(test_dir, "good\\Cam3P1L1\\obj3")
test_defective_dir = os.path.join(test_dir, "defective\\Cam3P1L1\\obj2")

# Define the class names to be predicted
class_names = ["good\\Cam3P1L1\\obj2", "defective\\Cam3P1L1\\obj2"]

img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

train_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
train_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
train_imgs = train_good_imgs + train_defective_imgs
train_labels = [0] * len(train_good_imgs) + [1] * len(train_defective_imgs)
train_labels = to_categorical(train_labels, num_classes=2) # convert to categorical labels
train_set = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
train_set = train_set.shuffle(len(train_imgs))
train_set = train_set.map(lambda x, y: (tf.io.read_file(x), y))
train_set = train_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
train_set = train_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
train_set = train_set.batch(batch_size)

val_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
val_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
val_imgs = val_good_imgs + val_defective_imgs
val_labels = [0] * len(val_good_imgs) + [1] * len(val_defective_imgs)
val_labels = to_categorical(val_labels, num_classes=2) # convert to categorical labels
val_set = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))
val_set = val_set.shuffle(len(val_imgs))
val_set = val_set.map(lambda x, y: (tf.io.read_file(x), y))
val_set = val_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
val_set = val_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
val_set = val_set.batch(batch_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model with categorical cross-entropy loss
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

# Evaluate the model on test data
test_good_imgs = [os.path.join(test_good_dir, f) for f in os.listdir(test_good_dir) if os.path.isfile(os.path.join(test_good_dir, f))]
test_defective_imgs = [os.path.join(test_defective_dir, f) for f in os.listdir(test_defective_dir) if os.path.isfile(os.path.join(test_defective_dir, f))]
test_imgs = test_good_imgs + test_defective_imgs
test_labels = [0] * len(test_good_imgs) + [1] * len(test_defective_imgs)
test_labels = to_categorical(test_labels, num_classes=2)
test_set = tf.data.Dataset.from_tensor_slices((test_imgs, test_labels))
test_set = test_set.shuffle(len(test_imgs))
test_set = test_set.map(lambda x, y: (tf.io.read_file(x), y))
test_set = test_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
test_set = test_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
test_set = test_set.batch(batch_size)

test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

Found 192 images belonging to 2 classes.
Epoch 1/30
3/3 [==============================] - 9s 2s/step - loss: 445.6335 - accuracy: 0.5116 - val_loss: 536.2238 - val_accuracy: 0.7442
Epoch 2/30
3/3 [==============================] - 6s 2s/step - loss: 614.6183 - accuracy: 0.7442 - val_loss: 232.7298 - val_accuracy: 0.2558
Epoch 3/30
3/3 [==============================] - 6s 2s/step - loss: 272.1339 - accuracy: 0.6047 - val_loss: 70.3107 - val_accuracy: 0.2558
Epoch 4/30
3/3 [==============================] - 6s 2s/step - loss: 160.6111 - accuracy: 0.5814 - val_loss: 10.2774 - val_accuracy: 0.7442
Epoch 5/30
3/3 [==============================] - 6s 2s/step - loss: 94.0721 - accuracy: 0.5814 - val_loss: 2.3344 - val_accuracy: 0.4186
Epoch 6/30
3/3 [==============================] - 6s 2s/step - loss: 105.8957 - accuracy: 0.5581 - val_loss: 39.9451 - val_accuracy: 0.7442
Epoch 7/30
3/3 [==============================] - 6s 2s/step - loss: 78.2931 - accuracy: 0.6977 - val_loss: 4.6506 - va

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI_image_IK15HGA_KUUE00E\\data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Set up the file paths for good and defective training images
train_good_dir = os.path.join(train_dir, "good\\Cam1P1L1\\obj4")
train_defective_dir = os.path.join(train_dir, "defective\\Cam1P1L1\\obj4")

# Set up the file paths for good and defective test images
test_good_dir = os.path.join(test_dir, "good\\Cam1P1L1\\obj4")
test_defective_dir = os.path.join(test_dir, "defective\\Cam1P1L1\\obj4")

# Define the class names to be predicted
class_names = ["good\\Cam1P1L1\\obj4", "defective\\Cam1P1L1\\obj4"]

img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

train_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
train_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
train_imgs = train_good_imgs + train_defective_imgs
train_labels = [0] * len(train_good_imgs) + [1] * len(train_defective_imgs)
train_labels = to_categorical(train_labels, num_classes=2) # convert to categorical labels
train_set = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
train_set = train_set.shuffle(len(train_imgs))
train_set = train_set.map(lambda x, y: (tf.io.read_file(x), y))
train_set = train_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
train_set = train_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
train_set = train_set.batch(batch_size)

val_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
val_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
val_imgs = val_good_imgs + val_defective_imgs
val_labels = [0] * len(val_good_imgs) + [1] * len(val_defective_imgs)
val_labels = to_categorical(val_labels, num_classes=2) # convert to categorical labels
val_set = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))
val_set = val_set.shuffle(len(val_imgs))
val_set = val_set.map(lambda x, y: (tf.io.read_file(x), y))
val_set = val_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
val_set = val_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
val_set = val_set.batch(batch_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model with categorical cross-entropy loss
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

# Evaluate the model on test data
test_good_imgs = [os.path.join(test_good_dir, f) for f in os.listdir(test_good_dir) if os.path.isfile(os.path.join(test_good_dir, f))]
test_defective_imgs = [os.path.join(test_defective_dir, f) for f in os.listdir(test_defective_dir) if os.path.isfile(os.path.join(test_defective_dir, f))]
test_imgs = test_good_imgs + test_defective_imgs
test_labels = [0] * len(test_good_imgs) + [1] * len(test_defective_imgs)
test_labels = to_categorical(test_labels, num_classes=2)
test_set = tf.data.Dataset.from_tensor_slices((test_imgs, test_labels))
test_set = test_set.shuffle(len(test_imgs))
test_set = test_set.map(lambda x, y: (tf.io.read_file(x), y))
test_set = test_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
test_set = test_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
test_set = test_set.batch(batch_size)

test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI_image_IK15HGA_KUUE00E\\data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Set up the file paths for good and defective training images
train_good_dir = os.path.join(train_dir, "good\\Cam1P1L1\\obj4")
train_defective_dir = os.path.join(train_dir, "defective\\Cam1P1L1\\obj4")

# Set up the file paths for good and defective test images
test_good_dir = os.path.join(test_dir, "good\\Cam1P1L1\\obj4")
test_defective_dir = os.path.join(test_dir, "defective\\Cam1P1L1\\obj4")

# Define the class names to be predicted
class_names = ["good\\Cam1P1L1\\obj4", "defective\\Cam1P1L1\\obj4"]

img_size = 224
batch_size = 16

# Define data augmentation techniques
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

train_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
train_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
train_imgs = train_good_imgs + train_defective_imgs
train_labels = [0] * len(train_good_imgs) + [1] * len(train_defective_imgs)
train_labels = to_categorical(train_labels, num_classes=2) # convert to categorical labels
train_set = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
train_set = train_set.shuffle(len(train_imgs))
train_set = train_set.map(lambda x, y: (tf.io.read_file(x), y))
train_set = train_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
train_set = train_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
train_set = train_set.batch(batch_size)

val_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
val_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
val_imgs = val_good_imgs + val_defective_imgs
val_labels = [0] * len(val_good_imgs) + [1] * len(val_defective_imgs)
val_labels = to_categorical(val_labels, num_classes=2) # convert to categorical labels
val_set = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))
val_set = val_set.shuffle(len(val_imgs))
val_set = val_set.map(lambda x, y: (tf.io.read_file(x), y))
val_set = val_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
val_set = val_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
val_set = val_set.batch(batch_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model with categorical cross-entropy loss
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=30,
    validation_data=val_set
)

# Evaluate the model on test data
test_good_imgs = [os.path.join(test_good_dir, f) for f in os.listdir(test_good_dir) if os.path.isfile(os.path.join(test_good_dir, f))]
test_defective_imgs = [os.path.join(test_defective_dir, f) for f in os.listdir(test_defective_dir) if os.path.isfile(os.path.join(test_defective_dir, f))]
test_imgs = test_good_imgs + test_defective_imgs
test_labels = [0] * len(test_good_imgs) + [1] * len(test_defective_imgs)
test_labels = to_categorical(test_labels, num_classes=2)
test_set = tf.data.Dataset.from_tensor_slices((test_imgs, test_labels))
test_set = test_set.shuffle(len(test_imgs))
test_set = test_set.map(lambda x, y: (tf.io.read_file(x), y))
test_set = test_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
test_set = test_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
test_set = test_set.batch(batch_size)

test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical



for i in range(3):
    for j in range(3):
        for k in range(3):
            # Set up the data directory path containing images and labels
            data_dir = "D:\\01.Paul\\AIVI_image_IK15HGA_KUUE00E\\data"
            train_dir = os.path.join(data_dir, "train")
            test_dir = os.path.join(data_dir, "test")

            # Set up the file paths for good and defective training images
            train_good_dir = os.path.join(train_dir, "good\\Cam1P1L"+str(j)+"\\obj" + str(k))
            train_defective_dir = os.path.join(train_dir, "defective\\Cam1P1L"+str(j)+"\\obj" + str(k))

            # Set up the file paths for good and defective test images
            test_good_dir = os.path.join(test_dir, "good\\Cam1P1L1\\obj" + str(k))
            test_defective_dir = os.path.join(test_dir, "defective\\Cam1P1L"+str(j)+"\\obj" + str(k))

            # Define the class names to be predicted
            class_names = ["good\\Cam1P1L1\\obj" + str(k), "defective\\Cam1P1L"+str(j)+"\\obj" + str(k)]

            img_size = 24
            batch_size = 16

            # Define data augmentation techniques
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                horizontal_flip=True,
                vertical_flip=True,
                zoom_range=0.2,
                validation_split=0.2,
            )

            train_set = train_datagen.flow_from_directory(
                train_dir,
                target_size=(img_size, img_size),
                batch_size=batch_size,
                class_mode='categorical',
                subset='training'
            )

            train_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
            train_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
            train_imgs = train_good_imgs + train_defective_imgs
            train_labels = [0] * len(train_good_imgs) + [1] * len(train_defective_imgs)
            train_labels = to_categorical(train_labels, num_classes=2) # convert to categorical labels
            train_set = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
            train_set = train_set.shuffle(len(train_imgs))
            train_set = train_set.map(lambda x, y: (tf.io.read_file(x), y))
            train_set = train_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
            train_set = train_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
            train_set = train_set.batch(batch_size)

            val_good_imgs = [os.path.join(train_good_dir, f) for f in os.listdir(train_good_dir) if os.path.isfile(os.path.join(train_good_dir, f))]
            val_defective_imgs = [os.path.join(train_defective_dir, f) for f in os.listdir(train_defective_dir) if os.path.isfile(os.path.join(train_defective_dir, f))]
            val_imgs = val_good_imgs + val_defective_imgs
            val_labels = [0] * len(val_good_imgs) + [1] * len(val_defective_imgs)
            val_labels = to_categorical(val_labels, num_classes=2) # convert to categorical labels
            val_set = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))
            val_set = val_set.shuffle(len(val_imgs))
            val_set = val_set.map(lambda x, y: (tf.io.read_file(x), y))
            val_set = val_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
            val_set = val_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
            val_set = val_set.batch(batch_size)

            model = tf.keras.models.Sequential([
                tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(2, activation='softmax')
            ])

            # Compile the model with categorical cross-entropy loss
            model.compile(loss='categorical_crossentropy',
                          optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
                          metrics=['accuracy'])

            # Train the model
            history = model.fit(
                train_set,
                epochs=30,
                validation_data=val_set
            )

            # Evaluate the model on test data
            test_good_imgs = [os.path.join(test_good_dir, f) for f in os.listdir(test_good_dir) if os.path.isfile(os.path.join(test_good_dir, f))]
            test_defective_imgs = [os.path.join(test_defective_dir, f) for f in os.listdir(test_defective_dir) if os.path.isfile(os.path.join(test_defective_dir, f))]
            test_imgs = test_good_imgs + test_defective_imgs
            test_labels = [0] * len(test_good_imgs) + [1] * len(test_defective_imgs)
            test_labels = to_categorical(test_labels, num_classes=2)
            test_set = tf.data.Dataset.from_tensor_slices((test_imgs, test_labels))
            test_set = test_set.shuffle(len(test_imgs))
            test_set = test_set.map(lambda x, y: (tf.io.read_file(x), y))
            test_set = test_set.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
            test_set = test_set.map(lambda x, y: (tf.image.resize(x, (img_size, img_size)), y))
            test_set = test_set.batch(batch_size)

            test_loss, test_acc = model.evaluate(test_set)
            print("Test accuracy:", test_acc)

Found 4052 images belonging to 2 classes.
Epoch 1/30
